In [122]:
import numpy as np
import Sudoku
import generator
import time
from copy import deepcopy
import gc
from IPython.display import display, clear_output

import importlib
importlib.reload(Sudoku)
importlib.reload(generator)

<module 'generator' from '/Users/giulio/Documents/sudoku/generator.py'>

In [10]:
# pick the next cell to assign a value to
def pick_next_cell(sudoku_table: Sudoku.Table):
    """Pick the next cell to assign a value to. The cell with the least number of possible values is picked.
    
    Parameters:
    sudoku_table (Sudoku.Table): The sudoku table to be solved.
    Returns:
    (int, int): The position of the cell with the least number of possible values.
    """
    min_cell = None
    min_cell_count = 10
    for row in range(9):
        for col in range(9):
            if sudoku_table.value[row, col] == 0:
                cell_count = len(sudoku_table.getPossibleValues(row, col))
                if cell_count < min_cell_count:
                    min_cell = (row, col)
                    min_cell_count = cell_count
    return min_cell

In [109]:
def forward_checking(sudoku_table: Sudoku.Table, row, col):
    """
    Performs forward checking on the given cell.

    Parameters:
    sudoku_table (Sudoku.Table): the sudoku table
    row (int): the row of the cell
    col (int): the column of the cell

    Returns:
    bool: True if the forward checking was successful, False otherwise
    """
    value = sudoku_table.value[row, col]
    neighbors = sudoku_table.neighbors(row, col)
    # consider only empty cells
    neighbors = [neighbor for neighbor in neighbors if sudoku_table.value[neighbor] == 0]
    
    for neighbor in neighbors:
        """ # we make a copy of the possible values, because we will modify the original
        possible_values = sudoku_table.getPossibleValues(neighbor[0], neighbor[1]).copy() """
        possible_values = sudoku_table.getPossibleValues(neighbor[0], neighbor[1])
        if (value in possible_values):
            possible_values.remove(value)
            #print("forward checking: removing value {} from cell {}".format(value, neighbor))
            if (len(possible_values) == 0):
                return False
            
    return True
        

In [96]:
def getBoxSet():
    """
    Create a set of the 9 boxes, where each box is a set of 9 tuples
    each tuple is the coordinates of a cell in the box

    Returns:
        set of boxes, where each box is a set of 9 tuples
    """
    box_set = []
    for i in range(3):
        for j in range(3):
            box = []
            for k in range(3):
                for l in range(3):
                    box.append((i*3+k, j*3+l))
            box_set.append(box)
    return box_set

def getArcsList(empty_cells):
    """
    Create a list of arcs as the cartesian product of the empty cells that are neighbors
    of each other
    
    Parameters:
        empty_cells: list of tuples (row, col)
    Returns:
        list of arcs, where each arc is a list of two tuples (row, col)"""
    arcs = []
    # for each row, define the cartesian product of the empty cells in that row
    # and add it to the arcs list
    for i in range(9):
        cells = [cell for cell in empty_cells if cell[0] == i]
        for cell1 in cells:
            for cell2 in cells:
                if (cell1 != cell2):
                    arcs.append([cell1, cell2])
    # we do the same for the columns
    for i in range(9):
        cells = [cell for cell in empty_cells if cell[1] == i]
        for cell1 in cells:
            for cell2 in cells:
                if (cell1 != cell2):
                    arcs.append([cell1, cell2])
    # and for the squares
    box_set = getBoxSet()
    for box in box_set:
        cells = [cell for cell in empty_cells if cell in box]
        for cell1 in cells:
            for cell2 in cells:
                if (cell1 != cell2):
                    #print("try to append arc " + str(cell1) + " " + str(cell2))
                    arcs.append([cell1, cell2])

    return arcs


def removeInconsistentValues(sudoku_table: Sudoku.Table, cell1, cell2):
    """
    Remove values from cell1's domain that are inconsistent with cell2's domain

    Parameters:
        cell1, cell2: tuples (row, col)
    Returns:
        True if a value is removed from cell1's domain, False otherwise
    """

    possible_values1 = sudoku_table.getPossibleValues(cell1[0], cell1[1])
    possible_values2 = sudoku_table.getPossibleValues(cell2[0], cell2[1])
    for v in possible_values1:
        if (len(possible_values2) == 1 and v == possible_values2[0]):
            sudoku_table.removePossibleValue(cell1[0], cell1[1], v)
            #print("AC3: removed value {} from cell {}".format(v, cell1))
            if (len(possible_values1) == 0):
                return True
    return False
            


def AC3(sudoku_table: Sudoku.Table):
    """
    AC3 algorithm for arc consistency

    Parameters:
        sudoku_table: the sudoku table

    Returns:
        True if the sudoku is arc-consistent, False otherwise
    """
    # get the indices of the empty cells
    empty_cells = sudoku_table.getEmptyCells()
    
    # create a list of arcs as the cartesian product of the empty cells that are neighbors
    # of each other
    arcs = getArcsList(empty_cells)
    for arc in arcs:
        if (removeInconsistentValues(sudoku_table, arc[0], arc[1])):
            return False
        
    return True




In [125]:
def csp_backtracking(sudoku_table: Sudoku.Table, start_time):
    """
    Solve a sudoku table using backtracking algorithm.

    Parameters:
    sudoku_table (Sudoku.Table): The sudoku table to be solved.
    start_time (float): the time when the algorithm started

    Returns:
    Sudoku.Table: The solved sudoku table, or False if the sudoku is not solvable.
    """
    # set a timeout of 10 seconds to avoid waiting too long
    """ if (time.time() - start_time > 100):
        return False """
    if (sudoku_table.isComplete()):
        return sudoku_table
    # run AC-3 and reduce var domains of unassigned variables; return failure if there exists a variable with empty set
    if (not AC3(sudoku_table)):
        #print(sudoku_table.value)
        #print("AC3 failed")
        return False
    """ else:
        clear_output(wait=True)
        print(sudoku_table.value) """
    
    cell_row, cell_col = pick_next_cell(sudoku_table)
    for value in sudoku_table.getPossibleValues(cell_row, cell_col):
        sudoku_table.assign(cell_row, cell_col, value)
        # We copy the sudoku table to keep track of the possible values,     
        s = deepcopy(sudoku_table)
        if (forward_checking(s, cell_row, cell_col)):
            #print("Assigned value", value, "to cell", cell_row, cell_col)
            result = csp_backtracking(s, start_time)
            if (result):
                #print("result found")
                return result
        #else:
            #backtrack
            # implicit by reassignment to the same cell: sudoku_table.backtrackAssignment(cell_row, cell_col)
    return False
        

In [118]:
def initial_pruning(sudoku_table: Sudoku.Table):
    """
    Prune the initial domains of the variables, considering non empty cells

    Parameters:
        sudoku_table: the sudoku table
    
    Returns:
        the sudoku table with the pruned domains
    """
    for row in range(9):
        for col in range(9):
            value = sudoku_table.value[row, col]
            if (value != 0):
                # remove the value from the possible values of the neighbors
                neighbors = sudoku_table.neighbors(row, col)
                # consider only empty cells
                neighbors = [neighbor for neighbor in neighbors if sudoku_table.value[neighbor] == 0]
                for neighbor in neighbors:
                    if (value in sudoku_table.getPossibleValues(neighbor[0], neighbor[1])):
                        #print("initial pruning: removing value {} from cell {}".format(value, neighbor))
                        sudoku_table.removePossibleValue(neighbor[0], neighbor[1], value)
    return sudoku_table

In [127]:
# try to solve a sudoku table
table = generator.generate_sudoku()
print(table.value)
table = initial_pruning(table)
start_time = time.time()
result = csp_backtracking(table, start_time)
print("\n\n")
print(result.value)

# create a new random sudoku table by removing some values from the previous one
table = generator.generate_sudoku_from_table(result)    

print(table.value)

[[0 0 0 6 0 0 8 0 0]
 [0 0 9 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0]
 [0 7 0 0 0 0 0 5 9]
 [0 0 0 8 0 0 0 0 4]
 [6 0 0 0 0 0 0 3 0]
 [1 0 0 0 0 0 2 0 0]
 [7 0 0 0 9 0 0 0 0]
 [0 0 0 4 5 0 0 0 0]]



[[4 3 7 6 2 9 8 1 5]
 [2 1 9 5 8 7 4 6 3]
 [8 5 6 3 1 4 9 7 2]
 [3 7 8 1 4 2 6 5 9]
 [5 9 1 8 6 3 7 2 4]
 [6 2 4 9 7 5 1 3 8]
 [1 4 5 7 3 8 2 9 6]
 [7 8 3 2 9 6 5 4 1]
 [9 6 2 4 5 1 3 8 7]]


In [99]:
def make_markdown_table(array):
    """ the same input as above """

    nl = "\n"

    markdown = nl
    markdown += f"| {' | '.join(array[0])} |"

    markdown += nl
    markdown += f"| {' | '.join(['---']*len(array[0]))} |"

    markdown += nl
    for entry in array[1:]:
        markdown += f"| {' | '.join(entry)} |{nl}"

    return markdown

In [101]:
make_markdown_table(result.value.astype(str).tolist())

'\n| 4 | 3 | 5 | 2 | 6 | 9 | 7 | 8 | 1 |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| 6 | 8 | 2 | 5 | 7 | 1 | 4 | 9 | 3 |\n| 1 | 9 | 7 | 8 | 3 | 4 | 5 | 6 | 2 |\n| 8 | 2 | 6 | 1 | 9 | 5 | 3 | 4 | 7 |\n| 3 | 7 | 4 | 6 | 8 | 2 | 9 | 1 | 5 |\n| 9 | 5 | 1 | 7 | 4 | 3 | 6 | 2 | 8 |\n| 5 | 1 | 9 | 3 | 2 | 6 | 8 | 7 | 4 |\n| 2 | 4 | 8 | 9 | 5 | 7 | 1 | 3 | 6 |\n| 7 | 6 | 3 | 4 | 1 | 8 | 2 | 5 | 9 |\n'

\n| 4 | 3 | 5 | 2 | 6 | 9 | 7 | 8 | 1 |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| 6 | 8 | 2 | 5 | 7 | 1 | 4 | 9 | 3 |\n|1 | 9 | 7 | 8 | 3 | 4 | 5 | 6 | 2 |\n| 8 | 2 | 6 | 1 | 9 | 5 | 3 | 4 | 7 |\n| 3 | 7 | 4 | 6 | 8 | 2 | 9 | 1 | 5 |\n| 9 | 5 | 1 | 7 | 4 | 3 | 6 | 2 | 8 |\n| 5 | 1 | 9 | 3 | 2 | 6 | 8 | 7 | 4 |\n| 2 | 4 | 8 | 9 | 5 | 7 | 1 | 3 | 6 |\n| 7 | 6 | 3 | 4 | 1 | 8 | 2 | 5 | 9 |\n